# Titanic: ML para predecir la Supervivencia

Este notebook es una guía a seguir para generar y utilizar un algoritmo de ML (**M**achine **L**earning) en la predicción de datos. 

En concreto, nuestro objetivo será realizar una clasificación binaria de la supervivencia o no de la gente que había a bordo del titanic en su hundimiento (**0**: La persona no sobrevive al hundimiento del titanic; **1**: sí que sobrevive). Basaremos esta clasificación en diferentes datos referentes a los ocupantes del barco (Referirse a la página: https://www.kaggle.com/c/titanic/data para obtener información sobre los campos de nuestras tablas)

## Primer Paso: Cargamos los Datos en tablas

Para este primer paso, utilizaremos una librería muy útil de python, con varias funciones interesantes que ya vienen implementadas: Pandas.

Tenemos que cargar 2 datasets, uno que nos servirá para entrenar nuestro modelo, que contendrá la variable que queremos predecir; y otro que servirá para probar el modelo y cuyos resultados tendremos que subir a la competición Kaggle.

In [105]:
# Importamos librerias necesarias
import pandas as pd, numpy as np

path = 'C:/Users/felix.hernandez/Desktop/Datathon_II/'

# Cargamos los dos sets de datos en dos Dataframes de Pandas.
test_df = pd.read_csv(path + 'test.csv', sep = ',')
train_df = pd.read_csv(path + 'train.csv', sep = ',')

## Exploración de los Datos

Ahora es el momento de movernos un poco por los datasets para familarizarnos con ellos. Normalmente, el *testing dataset* (usado para probar el modelo) tendrá una columna menos que el *training dataset* (el usado para entrenar el modelo), que será nuestra variable/columna **Objectivo** (o **Target**). Vamos a ver cuál es en nuestro caso:

In [106]:
# Obtenemos la lista de nombres de columna en el training y testing datasets
tr_c = train_df.columns
te_c = test_df.columns

# Buscamos diferencias entre las columnas: 2 formas de hacerlo
#target = []
#for col in tr_c:
#    if col not in te_c:
#        target.append(col)
        
# Manera más fácil, corta y rápida, la más "pythonica" :)
target = set(tr_c) - set(te_c)

print 'Las columnas presentes en ambos datasets son: %s \n' % (', '.join(tr_c))
print 'Nuestra(s) variable(s) objetivo es: %s' % (', '.join(target))

Las columnas presentes en ambos datasets son: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked 

Nuestra(s) variable(s) objetivo es: Survived


A la hora de entrenar un modelo de ML, hay que tener en cuenta dos cosas principalmente: 

1. Hay que diferenciar entre **features**, características o variables del modelo (las que se usan para predecir) y **target variable** que sería la variable a predecir

2. Nuestras *features* pueden ser de dos tipos: **Numéricas**, **Categóricas** o **Ordinales**. Diferencias entre estas variables:
 2. **Categóricas**: Tienen varias categorías (como su nombre indica) y cada  valor de la variable está dentro de una de estas categorías. No tiene sentido calcular medias o máximos/mínimos con ellas, pero sí calcular la frecuencia de aparición de cada categoría (por ejemplo)
 2. **Numéricas**: Pueden ser *continuas* o *discretas*. Toman valores numéricos entre un máximo y un mínimo. Con ellas tiene sentido calcular máximos, mínimos, medias... 
 2. **Ordinales**: Son similares a las variables categóricas, pero tienen un orden relativo entre ellas. Suelen tomar valores numéricos discretos (como su nombre indica)
    
Es el momento de visualizar los datos, para ver la pinta que tienen y sacar algunas conclusiones. Tenemos dos maneras de ver nuestros *Dataframes*: el método *sample(lineas_a_mostrar)*, o el método *head(lineas_a_mostrar)*. Usaremos el método sample() ya que la visualización es un poco mejor y comentaremos el otro.

In [107]:
print train_df.dtypes

train_df.sample(n=5)

# train_df.head(5)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.550,C22 C26,S
823,824,1,3,"Moor, Mrs. (Beila)",female,27.0,0,1,392096,12.475,E121,S
565,566,0,3,"Davies, Mr. Alfred J",male,24.0,2,0,A/4 48871,24.150,NaN,S
708,709,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,113781,151.550,NaN,S
805,806,0,3,"Johansson, Mr. Karl Johan",male,31.0,0,0,347063,7.775,NaN,S


De esta visualización podemos sacar algunas conclusiones:
* **Features Numéricas**: Age (continua), Fare(continua), SibSp(Discrete), Parch(Discrete)
* **Features Categóricas**: Survived, Sex, Embarked, Pclass. También podriamos incluir Ticket y Cabin, aunque realmente no tienen pinta de ser categorías si no un *carácter alfanumérico* (viendo esto podemos empezar a suponer que no son demasiado significativas para nosotros, pero lo veremos más adelante)

También nos hacemos una idea del tipo que son cada una de nuestras variables:
* Survived: int
* Pclass: int 
* Name: string
* Sex: string
* Age: float
* SibSp: int
* Parch: int
* Ticket: string
* Fare: float
* Cabin: string
* Embarked: string

También podemos llevar a cabo un análisis estadístico de nuestros datasets con el comando usado previamente:

In [108]:
print 'El número de pasajeros en nuestro Training Dataset es de: %s \n' % (train_df.shape[0])

train_df.describe(include = "all")

El número de pasajeros en nuestro Training Dataset es de: 891 



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Graham, Mr. George Edward",male,NaN,NaN,NaN,CA. 2343,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


Gracias a este resumen estadístico, podemos sacar varias conclusiones:

* Tenemos un total de 891 pasajeros en nuestro training dataset.
* Hay algunas variables que no están informadas para todos los pasajeros:
    * **Age**: $\frac{(891-714)}{891} \times 100 = 19.8$% de campos no informados
    * **Cabin**: $\frac{(891-204)}{891} \times 100 = 77.1$% de campos no informados
    * **Embarked**: $\frac{(891-889)}{891} \times 100 = 0.22$% de campos no informados
 
Como hemos visto, nos hemos centrado sobretodo en investigar los valores nulos. ¿Por qué? Sencillamente porque son los valores que más problemas dan a la hora de entrenar un modelo de ML. La mejor manera de ver los valores nulos es comprobar si són nulos y contar los que lo sean usando pandas:

In [109]:
print 'Número de nulos por campo en la dataframe de entrenamiento \n'
print pd.isnull(train_df).sum(), '\n'
print 'Número de nulos por campo en la dataframe de test \n'
print test_df.isnull().sum()

Número de nulos por campo en la dataframe de entrenamiento 

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64 

Número de nulos por campo en la dataframe de test 

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


Podemos hacernos una idea de lo importantes que son nuestras features analizando en detalle su relación con nuestra variable objetivo. Por ejemplo:

In [110]:
################################################################
######################## Para el SEXO ##########################
################################################################

# Mujeres que sobreviven:
# w_s = (train_df['Survived'][train_df.Sex == 'female'] == 1).count() 

# El método value_couns nos devuelve un cuent para cada valor de la columna seleccionada 
w_s = train_df['Survived'][train_df.Sex == 'female'].value_counts(normalize = True)[1]

# Hombres que sobreviven:
m_s = train_df['Survived'][train_df.Sex == 'male'].value_counts(normalize = True)[1]

print 'El porcentaje de mujeres que sobrevivieron fue de: {0}% \n'.format((w_s*100).round(2))
print 'El porcentaje de hombres que sobrevivieron fue de {0}% \n'.format((m_s*100).round(2))


################################################################
######################## Para Pclass ###########################
################################################################

# Tenemos tres valores para esta variable: 1, 2 y 3:
p_1 = train_df['Survived'][train_df.Pclass == 1].value_counts(normalize = True)[1]
p_2 = train_df['Survived'][train_df.Pclass == 2].value_counts(normalize = True)[1]
p_3 = train_df['Survived'][train_df.Pclass == 3].value_counts(normalize = True)[1]

print 'El porcentaje de pasajeros con Pclass = 1 que sobrevivieron fue de: {0}% \n'.format((p_1*100).round(2))
print 'El porcentaje de pasajeros con Pclass = 2 que sobrevivieron fue de: {0}% \n'.format((p_2*100).round(2))
print 'El porcentaje de pasajeros con Pclass = 3 que sobrevivieron fue de: {0}% \n'.format((p_3*100).round(2))


################################################################
########### Para Cabin (Informado o no informado) ##############
################################################################

# Crearemos una nueva variable CabinInf, un booleano que nos indicará si el pasajero
# tenía la cabina informada o no.
train_df['CabinInf'] = train_df.Cabin.notnull().astype(int) 
test_df['CabinInf'] = test_df.Cabin.notnull().astype(int)

inf_s = train_df[train_df['CabinInf'] == 1].Survived.value_counts(normalize = True)[1]
noinf_s = train_df[train_df['CabinInf'] == 0].Survived.value_counts(normalize = True)[1]

print 'El porcentaje de pasajeros con camarote informado que sobrevivieron fue de:    {0}% \n'.format((inf_s*100).round(2))
print 'El porcentaje de pasajeros con camarote no informado que sobrevivieron fue de: {0}% \n'.format((noinf_s*100).round(2))


El porcentaje de mujeres que sobrevivieron fue de: 74.2% 

El porcentaje de hombres que sobrevivieron fue de 18.89% 

El porcentaje de pasajeros con Pclass = 1 que sobrevivieron fue de: 62.96% 

El porcentaje de pasajeros con Pclass = 2 que sobrevivieron fue de: 47.28% 

El porcentaje de pasajeros con Pclass = 3 que sobrevivieron fue de: 24.24% 

El porcentaje de pasajeros con camarote informado que sobrevivieron fue de:    66.67% 

El porcentaje de pasajeros con camarote no informado que sobrevivieron fue de: 29.99% 



## Limpieza de los Datasets

Como vemos, en la última parte de la celda anterior hemos creado una nueva columna para estudiar el campo *Cabina*. Esta es una técnica bastante utilizada en ML cuando nos encontramos con campos con una gran cantidad de registros nulos, y que suele dar bastante buenos resultados.

Ahora que ya nos hemos hecho una idea de lo relacionadas que pueden estar nuestras variables a nuestra variable objetivo, vamos a **Limpiar los Datos** para poder entrenar nuestro modelo sin problemas.

Uno de los problemas más comunes es el tratamiento de los valores **nulos** dentro de nuestros datos. La mayoría de los modelos de ML no saben tratar los valores nulos, o sea que hay que idear una manera de tratar con ellos. Recordemos que tenemos 3 campos con valores nulos: Embarked (2 nulos), Age (177 nulos) y Cabin (687 nulos). Vamos a ver como tratar con ellos:

**IMPORTANTE**: Todos los cambios que se realicen en la Dataframe de **Train**, también deben realizarse en la df de **Test**!! 
Las dos dataframes deben ser iguales para que nuestro modelo pueda trabajar con ellas (mismas features). 

In [111]:
############################################################################
############################### EMBARKED ###################################
############################################################################

# Miramos cual és el puerto (de los 3 posibles) en el que embarca más gente
ports = train_df.groupby('Embarked'). count().PassengerId
# Puesto que hay 644 personas que embarcan en Southampton, podemos assignar los
# valores nulos a este puerto
# train_df.Embarked = train_df.Embarked.fillna('S')
train_df = train_df.fillna({'Embarked' : 'S'})

############################################################################
############################### CABIN ######################################
############################################################################
# Podemos hacer dos cosas con esta feature#: 
#     1 - O bien no la consideramos (no aporta casi información)
#     2 - Podemos utilizar el Booleano de antes (Informado o no)
# De momento no la consideramos, podemos probar a añadirla y ver si mejoran resultados
train_df.drop(['Cabin', 'CabinInf'], axis = 1, inplace = True)
test_df.drop(['Cabin', 'CabinInf'], axis = 1, inplace = True)


In [112]:
# train_df
############################################################################
############################### Age ########################################
############################################################################
# Para la edad es más difícil llenar huecos, ya que tenemos bastantes nulos. Cosas a tener en cuenta:
#    1. No podemos quitarla porque parece bastante significativa
#    2. No podemos usar la media para rellenar la edad, ya que no podemos asignarle a un niño
#       o a una persona mayor la media, que es de 29.699

# Presentaremos 2 propuestas. Cualquiera de las dos puede ser utilizada:
#    1. Agrupar en función del prefijo del nombre (Mr, Dr, Lady,...) y asignar el valor
#       de la media del grupo.
#    2. Usar crear una nueva variable que agrupe según el prefijo del nombre y no utilizar
#       la variable Age

# ------------------------------ PROPUESTA 1 ------------------------------------
train_df['Titulo'] = train_df.Name.str.extract('([A-Za-z]+)\.', expand = False) # extraer Prefijo
test_df['Titulo'] = test_df.Name.str.extract('([A-Za-z]+)\.', expand = False) # extraer Prefijo
    
# Vemos que hay algunos prefijos que no están bien escritos. Los corregimos:
train_df['Titulo'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
                               ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)
test_df['Titulo'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                               ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Miss'],inplace=True)

# Calculamos la edad media de cada grupo:
mean_ages = train_df.groupby('Titulo').Age.mean().round().astype(int)

# Por último, rellenamos los nulos (solo en train_df porque el test no tiene nulos en el campo Age):
for elem in mean_ages.index:
    train_df.loc[(train_df.Age.isnull()) & (train_df.Titulo == elem), 'Age'] = mean_ages[elem]
    test_df.loc[(test_df.Age.isnull()) & (test_df.Titulo == elem), 'Age'] = mean_ages[elem]
    
# train_df.isnull().sum()

In [113]:
# ------------------------------- Propuesta 2 -------------------------------------
# La dejamos comentada, solo hay que descomentarla. Hay muchas líneas de código porque hay que hacer algunos
# cálculos previos para poder usarla correctamente. 
'''
# Cálculos previos:
train_df["Age"] = train_df["Age"].fillna(-0.5)
test_df["Age"] = test_df["Age"].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
train_df['AgeGroup'] = pd.cut(train_df["Age"], bins, labels = labels)
test_df['AgeGroup'] = pd.cut(test_df["Age"], bins, labels = labels)

# Sacamos el prefijo del nombre de cada persona:
train_df['Titulo'] = train_df.Name.str.extract('([A-Za-z]+)\.', expand = False) # extraer Prefijo
test_df['Titulo'] = test_df.Name.str.extract('([A-Za-z]+)\.', expand = False) # extraer Prefijo

# Sustituimos los prefijos mal deletreados y añadimos alguno:
# Train
train_df['Titulo'].replace(['Lady', 'Capt', 'Col', 'Dr', 'Major', 'Rev',
   'Jonkheer', 'Dona'], 'Other', inplace = True)
train_df['Titulo'].replace('Don', 'Mr', inplace = True)    
train_df['Titulo'].replace(['Countess', 'Lady', 'Sir'], 'Royal', inplace = True)
train_df['Titulo'].replace(['Mlle', 'Ms'], 'Miss', inplace = True)
train_df['Titulo'].replace('Mme', 'Mrs', inplace = True)
# Test
test_df['Titulo'].replace(['Lady', 'Capt', 'Col', 'Dr', 'Major', 'Rev',
   'Jonkheer', 'Dona'], 'Other', inplace = True)
test_df['Titulo'].replace('Don', 'Mr', inplace = True)    
test_df['Titulo'].replace(['Countess', 'Lady', 'Sir'], 'Royal', inplace = True)
test_df['Titulo'].replace(['Mlle', 'Ms'], 'Miss', inplace = True)
test_df['Titulo'].replace('Mme', 'Mrs', inplace = True)


title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Royal": 5, "Other": 6}

train_df['Titulo'] = train_df.Titulo.map(title_mapping)
test_df['Titulo'] = test_df.Titulo.map(title_mapping)

mr_age = train_df[train_df["Titulo"] == 1]["AgeGroup"].mode() #Young Adult
miss_age = train_df[train_df["Titulo"] == 2]["AgeGroup"].mode() #Student
mrs_age = train_df[train_df["Titulo"] == 3]["AgeGroup"].mode() #Adult
master_age = train_df[train_df["Titulo"] == 4]["AgeGroup"].mode() #Baby
royal_age = train_df[train_df["Titulo"] == 5]["AgeGroup"].mode() #Adult
rare_age = train_df[train_df["Titulo"] == 6]["AgeGroup"].mode() #Adult

age_title_mapping = {1: "Young Adult", 2: "Student", 3: "Adult", 4: "Baby", 5: "Adult", 6: "Adult"}

train_df.loc[train_df.AgeGroup == 'Unknown', 'AgeGroup'] = train_df.Titulo.map(age_title_mapping)
test_df.loc[test_df.AgeGroup == 'Unknown', 'AgeGroup'] = test_df.Titulo.map(age_title_mapping)

age_mapping = {'Baby': 1, 'Child': 2, 'Teenager': 3, 'Student': 4, 'Young Adult': 5, 'Adult': 6, 'Senior': 7}

train_df['AgeGroup'] = train_df['AgeGroup'].map(age_mapping)
test_df['AgeGroup'] = test_df['AgeGroup'].map(age_mapping)

# Eliminamos la edad, usaremos AgeGroup:
train_df.drop('Age', axis = 1, inplace = True)
test_df.drop('Age', axis = 1, inplace = True)
'''

'\n# C\xc3\xa1lculos previos:\ntrain_df["Age"] = train_df["Age"].fillna(-0.5)\ntest_df["Age"] = test_df["Age"].fillna(-0.5)\nbins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]\nlabels = [\'Unknown\', \'Baby\', \'Child\', \'Teenager\', \'Student\', \'Young Adult\', \'Adult\', \'Senior\']\ntrain_df[\'AgeGroup\'] = pd.cut(train_df["Age"], bins, labels = labels)\ntest_df[\'AgeGroup\'] = pd.cut(test_df["Age"], bins, labels = labels)\n\n# Sacamos el prefijo del nombre de cada persona:\ntrain_df[\'Titulo\'] = train_df.Name.str.extract(\'([A-Za-z]+)\\.\', expand = False) # extraer Prefijo\ntest_df[\'Titulo\'] = test_df.Name.str.extract(\'([A-Za-z]+)\\.\', expand = False) # extraer Prefijo\n\n# Sustituimos los prefijos mal deletreados y a\xc3\xb1adimos alguno:\n# Train\ntrain_df[\'Titulo\'].replace([\'Lady\', \'Capt\', \'Col\', \'Dr\', \'Major\', \'Rev\',\n   \'Jonkheer\', \'Dona\'], \'Other\', inplace = True)\ntrain_df[\'Titulo\'].replace(\'Don\', \'Mr\', inplace = True)    \ntrain_df[\'Titulo\'].r

Ya no tenemos valores nulos en la dataframe que utlizaremos para entrenar el modelo. Vamos a limpiar el resto de columnas:

In [114]:
############################################################################
############################### Name #######################################
############################################################################
# Ya no nos sirve de nada, la hemos usado para arreglar la Edad pero no aporta información extra. La quitamos
train_df.drop('Name', axis = 1, inplace = True)
test_df.drop('Name', axis = 1, inplace = True)
    
############################################################################
########################## Ticket & Passenger Id ###########################
############################################################################
# También las eliminamos, ya que no aportan ninguna información útil al modelo
train_df.drop('Ticket', axis = 1, inplace = True)
test_df.drop('Ticket', axis = 1, inplace = True)

train_df.drop('PassengerId', axis = 1, inplace = True)
test_df.drop('PassengerId', axis = 1, inplace = True)

In [115]:
############################################################################
############################### Fare #######################################
############################################################################
# Tenemosn un valor nulo en Fare en nuestro dataset de Test. Le asignaremos el valor de la media de su PClass
# De nuevo, como con la edad, tenemos 2 opciones:
#     1. Utilizamos el Fare (precio del pasaje) con valores continuos
#     2. Creamos categorías para agrupar precios similares

# ------------------------------- Propuesta 1 -------------------------------------
p_class = test_df.loc[test_df.Fare.isnull(), 'Pclass'].values[0]
test_df.loc[test_df.Fare.isnull(), 'Fare'] = test_df.groupby('Pclass').Fare.mean()[p_class]

# ------------------------------- Propuesta 2 -------------------------------------
# Creamos 5 categorías para el precio en función de los cuantiles:
'''train_df['FareCat'] = pd.qcut(train_df['Fare'], 5, labels = [1, 2, 3, 4, 5])
test_df['FareCat'] = pd.qcut(test_df['Fare'], 5, labels = [1, 2, 3, 4, 5])

#drop Fare values
train_df = train.drop(['Fare'], axis = 1)
test_df = test.drop(['Fare'], axis = 1)'''

"train_df['FareCat'] = pd.qcut(train_df['Fare'], 5, labels = [1, 2, 3, 4, 5])\ntest_df['FareCat'] = pd.qcut(test_df['Fare'], 5, labels = [1, 2, 3, 4, 5])\n\n#drop Fare values\ntrain_df = train.drop(['Fare'], axis = 1)\ntest_df = test.drop(['Fare'], axis = 1)"

## Prediccion: Machine Learning con distintos algoritmos

LLega el momento de utilizar algoritmos para predecir la (no) supervivencia de nuestros pasajeros a bordo del titanic. Una libreria muy interesante de *python* en el ámbito del *Machine Learning* es **scikit-learn (sklearn)**. Esta librería contiene muchos algoritmos ya implementados, así como módulos para aplicar cambios necesarios en nuestros datos de forma más sencilla. Realizaremos la predicción usando diferentes algoritmos:

* Gaussian Naive Bayes
* Logistic Regression
* Support Vector Machine (SVM)
* Perceptron
* Decision Tree Classifier
* Random forest Classifier
* K-Nearest Neighbors
* Stochastic Gradient Descent
* Gradient Boosting Classifier

Como extra, realizaremos una clasificación con Gradient Boost Classifiers usando el módulo **XGBoost** para python.

Antes de ponernos a utilizar lo modelos, usaremos un par de módulos de sklearn para arreglar 2 cosas:
1. Estos algoritmos no admiten variables categóricas de tipo string. Solo admiten valores enteros o decimales. Por ello usaremos un *label encoder*. Lo que hace este label encoder básicamente es usar un diccionario para pasar nuestras variables categóricas a categorías numéricas.
2. Dividiremos los datos en **entrenamiento** y **validación**. Esto es muy importante, ya que para calcular la precisión del modelo necesitamos unos datos para comprobar nuestras predicciones. Habitualmente, se coge un 70/80% de los datos para *entrenar* el modelo, y un 20/30% para *validar* los resultados

In [116]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Parte 1: Codificar las variables categóricas
# Creamos un clasificador para cada variable categórica a codificar. Podemos crear solo uno para todas,
# pero entonces no nos guarda las categorías por si queremos invertir la codificación.
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()

# La manera más rapida:
train_df.Sex = le1.fit_transform(train_df.Sex)
test_df.Sex = le1.transform(test_df.Sex)

train_df.Embarked = le2.fit_transform(train_df.Embarked)
test_df.Embarked = le2.transform(test_df.Embarked)

train_df.Titulo = le3.fit_transform(train_df.Titulo)
test_df.Titulo = le3.transform(test_df.Titulo)

# Podemos ver las clases que contienen:
# print list(le1.classes_)
# print list(le2.classes_)

# Parte 2: Dividir los datos en training y testing:
predictors = train_df.drop(['Survived'], axis=1)
target = train_df["Survived"]

x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 12)

Llega el momento de aplicar los diferentes modelos a nuestros datos y ver qué resultados obtenemos. Como hemos dicho, utilizaremos la librería sklearn de python. Previo a la aplicación de cada modelo, habrá una breve explicación de la idea fundamental de cada modelo, de su funcionamiento a nivel más básico.

Antes que nada , notar que todos los algoritmos mencionados son **Algoritmos de Clasificación, no de Regresión**.


### Gaussian Naive Bayes

Este algoritmo de aprendizaje supervisado trabaja con probabilidades, usando el *Teorema de Bayes*. Calcula la probabilidad de que un elemento esté en cada una de las clases que tenemos. La clase con mayor probabilidad es la que se devuelve como predicción. Este algoritmo asume que las direfentes variables no están relacionadas entre sí, que todas contribuyen de igual manera al resultado de la clasificación.
Vamos a ver su implementación:

In [117]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gau = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Gaussian Naive Bayes\": {0} % '.format(acc_gau)


Precisión del algoritmo de clasificación "Gaussian Naive Bayes": 81.22 % 


### Logistic Regression Classifier

Este método de aprendizaje supervisado recibe su nombre de la función que utiliza en su implementación, la *función logística (logistic function)*:
$$ f(x) = \frac{1}{1+e^{-x}}$$

Asigna unos pesos o coeficientes a cada una de las variables que miden su importancia a la hora de predecir el resultado. La función utilizada al final es del tipo:
$$ P(X) = \frac{e^{(b_0 + b_1*X)}}{1 + e^{(b_0 + b_1*X)}} $$

que proporciona la probabilidad de estar en una determinada categoria. El modelo calcula los coeficientes $b_0$, $b_1$ y los ajusta a los datos para mejorar la precisión del modelo. Vamos a ver que tal funciona:

In [118]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_lreg = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Logistic Regression\": {0} % '.format(acc_lreg)

Precisión del algoritmo de clasificación "Logistic Regression": 78.17 % 


### Support Vector Machine

SVM es un algoritmo de aprendizaje supervisado para regresión/clasificación que se basa en encontrar el hiperplano que mejor divide los datos entre nuestras clases.
![alt text](https://66.media.tumblr.com/ff709fe1c77091952fb3e3e6af91e302/tumblr_inline_o9aa8dYRkB1u37g00_540.png "Hiperplane Example")

Los **vectores de soporte** son los más cercanos al hiperplano. Son aquellos que, de ser eliminados, cambiarían la disposición del hiperplano.

Este hiperplano se escoje utilizando *márgenes*: la distancia entre el hiperplano y el punto más cercano de cada clase. Se intenta encontrar un hiperplano con los mayores márgenes posibles, para facilitar la correcta clasificación de nuevos elementos.

![alt text](https://66.media.tumblr.com/7f12391977435370c1ddf4945dca0575/tumblr_inline_o9aa9nH3WQ1u37g00_540.png "Margins Example")

Esta es la idea general que hay detrás del algoritmo. Vamos a ver como funciona:

In [119]:
# Support Vector Machine
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
acc_svmc = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"SVM Classifier\": {0} % '.format(acc_svmc)

# Linear SVMC
from sklearn.svm import LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_val)
acc_lsvmc = round(accuracy_score(y_pred, y_val) * 100, 2)


print 'Precisión del algoritmo de clasificación \"LSVM Classifier\": {0} % '.format(acc_lsvmc)


Precisión del algoritmo de clasificación "SVM Classifier": 71.07 % 
Precisión del algoritmo de clasificación "LSVM Classifier": 62.44 % 


### Perceptron

Es un algoritmo de aprendizaje supervisado que sirve para entrenar un clasificador binario (como es nuestro caso). Mapea un vector de inputs $x$, a un valor output $f(x)$ usando la siguiente función:
$$f(x) = 
\begin{cases}
1, & \text{if } w · x + b > 0\\
0, & \text{otherwise}
\end{cases}
$$
donde $w$ es un vector de pesos (mide la importancia de las diferentes variables) y $w·x = \sum_{i = 1}^{m}w_i*x_i$ 

Por ejemplo, un caso de perceptrón muy básico sería:
<img src="https://appliedgo.net/media/perceptron/heaviside.png" width="400" height="200" />


Veamos que tal lo hace con nuestros datos:

In [120]:
# Perceptron
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(x_train, y_train)
y_pred = perceptron.predict(x_val)
acc_perceptron = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Perceptron\": {0} %'.format(acc_perceptron)

Precisión del algoritmo de clasificación "Perceptron": 56.35 %


### Decision tree

Como su nombre indica, este algoritmo de aprendizaje supervisado utiliza una estructura en forma de árbol para clasificar nuestros datos. La idea es colocar en la capa más alta del árbol (en la raíz) el **atributo** más significativo. A partir de este, se dividen los datos en dos ramas, que tendrán nuevos nodos en los que se volverán a dividir los datos en función de las demás variables o atributos. 

<img src="https://i1.wp.com/dataaspirant.com/wp-content/uploads/2017/01/Decision-tree-python.jpg?resize=350%2C200" width="400" height="200" />

Para más información: http://dataaspirant.com/2017/01/30/how-decision-tree-algorithm-works/

Veamos qué tal funcionan los árboles de decisión para nuestro Dataset.

In [121]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
acc_dtree = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Decision Tree\": {0} %'.format(acc_dtree)

Precisión del algoritmo de clasificación "Decision Tree": 76.65 %


### Random Forest

Es un algoritmo de aprendizaje supervisado de la familia de los *ensembled algorithms*. Estos algoritmos son todos aquellos que combinan algoritmos más simples para enerar un modelo más robusto. En el caso de un *Random Forest* se crean y combinan un conjunto de *Decision Trees* para conseguir una predicción mejor de la que conseguiríamos con los árboles de decisión. Ejemplo:

<img src="https://i.ytimg.com/vi/ajTc5y3OqSQ/hqdefault.jpg" width="400" height="200" />

Random forest es uno de los algoritmos utilizados por *XGBoost* un algoritmo muy potente utilizado en ML. Vamos a ver qué tal lo hacen con nuestros datos.

In [122]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_rf = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Random Forest\": {0} %'.format(acc_rf)

Precisión del algoritmo de clasificación "Random Forest": 80.71 %


### K-Nearest Neighbors (KNN)

Es un algoritmo de aprendizaje supervisado, de tipo *non parametric lazy learning*. *Non parametric* significa que asume ninguna característica especial de los datos que utilizamos para entrenarlo o evaluarlo. *Lazy learning* implica que no utiliza los datos de entrenamiento para hacer una generalización. Esto quiere decir que no hay una fase de entrenamiento explícita o que esta es mínima, lo que implica que la fase de entrenamiento es muy rápida.

Supongamos $k = 1$. La idea básica del algoritmo de KNN es la siguiente:

1. encontrar el punto más *"cercano"* al punto a evaluar dentro de nuestros puntos clasificados. 
2. Una vez encontrado, se le asigna al nuevo punto el valor de este vecino que hemos encontrado. 

Este razonamiento suele ser válido cuando el número de puntos a evaluar muy grande: dos puntos similares tendrán la misma etiqueta en el caso general.

Para el caso general, cuando $k = K$, se realiza una generalización del algoritmo anterior. Se encuentran los K vecinos más cercanos al punto a evaluar, y se asigna al punto en cuestión la etiqueta que aparezca más veces entre los vecinos encontrados. Parece lógico que la precisión del algoritmo mejora cuanto mayor es la $K$, pero también aumenta el coste computacional.

<img src="http://www.statsoft.com/portals/0/Support/KNNOverViewImageA.jpg" width="400" height="200" />

Veamos que tal funciona:


In [123]:
# KNN or k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_val)
acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"KNN\": {0} %'.format(acc_knn)

Precisión del algoritmo de clasificación "KNN": 66.5 %


### Stochastic Gradient Descent

Es un algoritmo de aprendizaje supervisado cuyo objetivo es minimizar una función (habitualmente llamada *loss function*). Dada una función que depende de unos determinados parámetros, *gradient descent* empieza con un set inicial de parámetros y avanza iterativamente hacia un valor de dichos parámetros que minimice la función objetivo. Esta minimización se consigue utilizando la función gradiente (derivando la función objetivo, utilizando la *pendiente*). 

<img src="https://qph.ec.quoracdn.net/main-qimg-b7a3a254830ac374818cdce3fa5a7f17" width="400" height="200" />

Veamos qué tal funciona:


In [124]:
# Stochastic Gradient Descent
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_val)
acc_sgd = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"KNN\": {0} %'.format(acc_sgd)

Precisión del algoritmo de clasificación "KNN": 53.3 %


### Gradient Boosting Classifier

Este algoritmo de aprendizaje es otro de los llamados *ensemble algorithms*. La idea detras del Gradient Boosting es la creación de muchos *weak learners*, clasificadores muy simples, que iterativamente se van juntando para mejorar las predicciones. En concreto, en este algoritmo suelen utilizarse árboles de decisión como week learners. Tiene 3 componentes principales:

1. Una función de pérdida (*loss function*) a optimizar
2. *Weak learners*
3. Un *modelo aditivo* con el que añadir nuevos weak learners a nuestro modelo

Se añade un árbol de decisión por iteración, utilizando un método de descenso del gradiente (algoritmo anterior) para minimizar la función de pérdida cuando se añade este nuevo árbol. En vez de parámetros, en cada iteración se debe añadir un árbol de decisión que, junto con los que ya tenemos en el modelo, reduzca la función de pérdida. El nuevo árbol deberá intentar *"corregir"* el error provocado por las predicciones de los árboles anteriores:

<img src="http://arogozhnikov.github.io/images/gbdt_attractive_picture.png" width="5000" height="500" />

Esta técnica también se utiliza (combinada con Random Forest) en el algoritmo de predicción XGBoost, que se plantea como extra para este DATATHON.
Veamos qué tal funciona:

In [125]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

gbk = GradientBoostingClassifier()
gbk.fit(x_train, y_train)
y_pred = gbk.predict(x_val)
acc_gbk = round(accuracy_score(y_pred, y_val) * 100, 2)
print 'Precisión del algoritmo de clasificación \"Gradient Boosting\": {0} %'.format(acc_gbk)

Precisión del algoritmo de clasificación "Gradient Boosting": 79.19 %


### XGBoost

## Conclusiones

Como hemos podido observar, hemos obtenido una idea de como entrenar y evaluar varios modelos de predicción con nuestros datos. A la hora de crear un algoritmo de Machine Learning, hay varias cosas que se deben tener en cuenta:

1. Debemos intentar evitar el **overfitting**: Entrenar el modelo con los mismos datos que lo evaluamos, puede resultar en la creación de un modelo poco general, que se ajusta muy bien a éstos datos concretos pero puede fallar al extenderlo a nuevos datos (sobretodo si son muy diferentes a los actuales). Una manera de evitarlo, es hacer un **random sampling**, partir nuestros datos de manera aleatoria de varias formas diferentes y usarlos para entrenar/validar el modelo varias veces.

2. Casi todos los modelos tienen parámetros (que aquí no hemos tenido en cuenta, hemos usado los definidos por defecto) que pueden ser ajustados para mejorar la predicción. Por ejemplo, en el caso del **XGBoost**, hay cerca de 20 parámetros que puden ser ajustados a voluntad y que pueden variar la calidad de la predicción.

3. Hay algunos modelos que trabajan mejor con sets de datos pequeños y similares, mientras que otros tienen mejor rendimiento para sets de datos grandes y muy dispares. Un algoritmo nos puede ir muy bien para clasificar una variable que depende de pocas *features*, pero no para variables que dependen de muchas.

Para hacernos una mejor idea del funcionamiento de los modelos, haremos varias particiones de los datos y calcularemos la precisión de los modelos para todas ellas. Finalmente, calcularemos una media de las puntuaciones obtenidas para poder decidir que modelo se ajusta mejor a nuestros datos:

In [126]:
results = pd.DataFrame({
    'Model' : ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 'Linear SVC', 
              'Decision Tree', 'Stochastic Gradient Descent', 'Gradient Boosting Classifier'],
    'Score_1': [acc_svmc, acc_knn, acc_lreg, 
              acc_rf, acc_gau, acc_perceptron,acc_lsvmc, acc_dtree,
              acc_sgd, acc_gbk]})

In [127]:
# Haremos 20 iteraciones para todos los modelos con particiones diferentes de los datos.
for i in range(2,20):
    if (i // 10 > 0):
        x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.25)

    else:
        x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.20)

# acc_svmc, acc_knn, acc_lreg, acc_randomforest, acc_gau, acc_perceptron,acc_lsvmc, acc_decisiontree,acc_sgd, acc_gbk
    # Calculamos todas las predicciones y todas las precisiones:
    
    # Gaussian Naive Bayes:
    gaussian.fit(x_train, y_train)
    y_pred = gaussian.predict(x_val)
    acc_gau = round(accuracy_score(y_pred, y_val) * 100, 2)

    # Logistic regression:
    logreg.fit(x_train, y_train)
    y_pred = logreg.predict(x_val)
    acc_lreg = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # SVMC y LSVMC:
    svc.fit(x_train, y_train)
    y_pred = svc.predict(x_val)
    acc_svmc = round(accuracy_score(y_pred, y_val) * 100, 2)

    linear_svc.fit(x_train, y_train)
    y_pred = linear_svc.predict(x_val)
    acc_lsvmc = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Perceptron:
    perceptron.fit(x_train, y_train)
    y_pred = perceptron.predict(x_val)
    acc_perceptron = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Decision tree:
    decisiontree.fit(x_train, y_train)
    y_pred = decisiontree.predict(x_val)
    acc_dtree = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Random Forest:
    randomforest.fit(x_train, y_train)
    y_pred = randomforest.predict(x_val)
    acc_rf = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # KNN:
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_val)
    acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Gradient Descent:
    sgd.fit(x_train, y_train)
    y_pred = sgd.predict(x_val)
    acc_sgd = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Lo metemos todo en la DF de resultados:
    results['Score_'+str(i)] = [acc_svmc, acc_knn, acc_lreg, 
              acc_rf, acc_gau, acc_perceptron,acc_lsvmc, acc_dtree,
              acc_sgd, acc_gbk]

Una vez calculados todos los scores para las diferentes iteraciones, calculamos la media de las puntuaciones y ordenamos de mejor a peor

In [129]:
results.index = results['Model'].values
results['Mean'] = results.mean(axis = 1)
conclusion = results[['Model', 'Mean']].sort_values(by = 'Mean', ascending = False)
conclusion

,Model,Mean
Random Forest,Random Forest,80.825263
Naive Bayes,Naive Bayes,80.416316
Logistic Regression,Logistic Regression,79.494737
Gradient Boosting Classifier,Gradient Boosting Classifier,79.190000
Decision Tree,Decision Tree,77.253684
Linear SVC,Linear SVC,73.168947
Support Vector Machines,Support Vector Machines,70.820526
KNN,KNN,70.303158
Perceptron,Perceptron,64.698421
Stochastic Gradient Descent,Stochastic Gradient Descent,61.127895


## Cosas Opcionales

Podemos probar varias técnicas diferentes con nuestros datos para ver si nuestras predicciones mejoran: Podemos normalizar los datos, incluir otras columnas (como ahora utilizar un booleano para las columnas que tengan muchos valores...)

In [130]:
# Normalizar:
train_norm = train_df.apply(lambda x: (x - np.mean(x)) / (np.std(x)))

In [131]:
# Haremos 20 iteraciones para todos los modelos con particiones diferentes de los datos.

predictors = train_norm.drop(['Survived'], axis=1)
target = train_df["Survived"]

x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.22, random_state = 12)

for i in range(2,20):
    if (i // 10 > 0):
        x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.25)

    else:
        x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.20)

# acc_svmc, acc_knn, acc_lreg, acc_randomforest, acc_gau, acc_perceptron,acc_lsvmc, acc_decisiontree,acc_sgd, acc_gbk
    # Calculamos todas las predicciones y todas las precisiones:
    
    # Gaussian Naive Bayes:
    gaussian.fit(x_train, y_train)
    y_pred = gaussian.predict(x_val)
    acc_gau = round(accuracy_score(y_pred, y_val) * 100, 2)

    # Logistic regression:
    logreg.fit(x_train, y_train)
    y_pred = logreg.predict(x_val)
    acc_lreg = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # SVMC y LSVMC:
    svc.fit(x_train, y_train)
    y_pred = svc.predict(x_val)
    acc_svmc = round(accuracy_score(y_pred, y_val) * 100, 2)

    linear_svc.fit(x_train, y_train)
    y_pred = linear_svc.predict(x_val)
    acc_lsvmc = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Perceptron:
    perceptron.fit(x_train, y_train)
    y_pred = perceptron.predict(x_val)
    acc_perceptron = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Decision tree:
    decisiontree.fit(x_train, y_train)
    y_pred = decisiontree.predict(x_val)
    acc_dtree = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Random Forest:
    randomforest.fit(x_train, y_train)
    y_pred = randomforest.predict(x_val)
    acc_rf = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # KNN:
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_val)
    acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Gradient Descent:
    sgd.fit(x_train, y_train)
    y_pred = sgd.predict(x_val)
    acc_sgd = round(accuracy_score(y_pred, y_val) * 100, 2)
    
    # Lo metemos todo en la DF de resultados:
    results['Score_'+str(i)] = [acc_svmc, acc_knn, acc_lreg, 
              acc_rf, acc_gau, acc_perceptron,acc_lsvmc, acc_dtree,
              acc_sgd, acc_gbk]

In [132]:
results.index = results['Model'].values
results['Mean'] = results.mean(axis = 1)
conclusion = results[['Model', 'Mean']].sort_values(by = 'Mean', ascending = False)
conclusion

,Model,Mean
Support Vector Machines,Support Vector Machines,81.165526
Random Forest,Random Forest,80.317263
Gradient Boosting Classifier,Gradient Boosting Classifier,79.190000
Logistic Regression,Logistic Regression,78.830737
Naive Bayes,Naive Bayes,78.750316
KNN,KNN,78.381158
Linear SVC,Linear SVC,77.487947
Decision Tree,Decision Tree,76.632184
Perceptron,Perceptron,71.220921
Stochastic Gradient Descent,Stochastic Gradient Descent,70.836895
